In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RedditNamedEntities") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()


24/04/12 23:28:24 WARN Utils: Your hostname, DESKTOP-S2RF5B5 resolves to a loopback address: 127.0.1.1; using 172.30.154.145 instead (on interface eth0)
24/04/12 23:28:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/arvind/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/arvind/.ivy2/cache
The jars for the packages stored in: /home/arvind/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-29738c09-862d-420b-8c05-cb2e27795532;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 616ms :: artifacts dl 21ms
	::

In [2]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "topic1") \
  .load()


In [3]:
from pyspark.sql.functions import col, expr
comments = df.selectExpr("CAST(value AS STRING)")


In [4]:
comments

DataFrame[value: string]

In [5]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Define a UDF to extract named entities and return them as an array of strings
@pandas_udf("array<string>", PandasUDFType.SCALAR)
def extract_entities(texts: pd.Series) -> pd.Series:
    entities = []
    for doc in nlp.pipe(texts):
        entities.append([ent.text for ent in doc.ents])
    return pd.Series(entities)


# Apply the UDF to each comment to extract entities
entities_df = comments.withColumn("named_entity", extract_entities(col("value")))



/home/arvind/.local/lib/python3.8/site-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [6]:
from pyspark.sql.functions import explode, count

exploded_entities = entities_df.select(explode(col("named_entity")).alias("entity"))
entity_counts = exploded_entities.groupBy("entity").count()


In [7]:
from pyspark.sql.functions import to_json, struct, col

# Convert the entity and count columns into a JSON string suitable for Kafka
kafka_output_df = entity_counts \
    .selectExpr("CAST(entity AS STRING) AS key", "to_json(struct(*)) AS value")


In [ ]:
kafka_output_query = kafka_output_df \
    .writeStream \
    .outputMode("update") \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "topic2") \
    .option("checkpointLocation", "/home/arvind/reddit_kafka_project/spark_checkpoints") \
    .start()

kafka_output_query.awaitTermination()


24/04/12 23:28:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/12 23:28:42 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
24/04/12 23:28:43 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
24/04/12 23:28:48 WARN HDFSBackedStateStoreProvider: The state for version 759 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
24/04/12 23:28:48 WARN HDFSBackedStateStoreProvider: The state for version 759 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
24/04/12 23:28:48 WARN HDFSBackedStateStoreP